In [1]:
from collections import defaultdict
import json
from pathlib import Path
import sys

import pandas as pd
from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
import numpy as np
import torch
import wandb

from chemprop import data, featurizers, models, nn

sys.path.insert(0, '../agenticadmet')
from eval import eval_admet, extract_preds, extract_refs

In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
NUM_WORKERS = 0 # number of workers for dataloader. 0 means using main process for data loading
SMILES_COLUMN = 'smiles_std'
TARGET_COLUMNS = ['LogHLM', 'LogMLM', 'LogD', 'LogKSOL', 'LogMDR1-MDCKII']

In [4]:
def prepare_data(input_df, smiles_column_for_pred: str = SMILES_COLUMN):
    train_data, val_data = [], []
    for _, row in input_df.iterrows():
        dp = data.MoleculeDatapoint.from_smi(row[SMILES_COLUMN], row[TARGET_COLUMNS].values)
        if row['split'] == 'train':
            train_data.append(dp)
        elif row['split'] == 'val':
            val_data.append(dp)

    pred_data = []
    for _, row in input_df.iterrows():
        dp = data.MoleculeDatapoint.from_smi(row[smiles_column_for_pred], row[TARGET_COLUMNS].values)
        pred_data.append(dp)

    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

    train_dset = data.MoleculeDataset(train_data, featurizer)
    # scaler = train_dset.normalize_targets()

    val_dset = data.MoleculeDataset(val_data, featurizer)
    # val_dset.normalize_targets(scaler)

    pred_dset = data.MoleculeDataset(pred_data, featurizer)

    return train_dset, val_dset, pred_dset

In [5]:
def train_model(config, train_dset, val_dset, num_workers, save_dir, run_idx, enable_logger=True):
    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])
    dropout = float(config["dropout"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth, dropout=dropout)
    agg = nn.MeanAggregation()
    ffn = nn.RegressionFFN(
        n_tasks=len(TARGET_COLUMNS),
        output_transform=None, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers,
        dropout=dropout
    )
    batch_norm = True
    metric_list = [nn.metrics.MAE(), nn.metrics.R2Score()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    ckpt_callback = ModelCheckpoint(
        save_top_k=0,
        save_last=True
    )

    if enable_logger:
        exp_name = f"chemprop_run_{run_idx}"
        logger = WandbLogger(
            project="admet-challenge",
            name=exp_name,
            prefix=f"{save_dir.stem}",
            save_dir=f"../wandb/{exp_name}"
        )
    else:
        logger = None

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=200,
        enable_progress_bar=False,
        callbacks=[ckpt_callback],
        default_root_dir=save_dir,
        logger=logger
    )

    try:
        trainer.fit(model, train_loader, val_loader)
    except Exception as e:
        if logger is not None:
            logger.finalize("failed")
            wandb.finish(exit_code=1)
        raise e
    else:
        if logger is not None:
            logger.finalize("success")

    return model

def predict(model, pred_dset, num_workers):
    pred_loader = data.build_dataloader(pred_dset, num_workers=num_workers, shuffle=False)
    
    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        enable_progress_bar=False
    )

    model.eval()
    preds = trainer.predict(model, pred_loader, return_predictions=True)
    preds = torch.cat(preds)

    return preds

In [6]:
MODEL_CONFIG = {
    "depth": 4,
    "ffn_hidden_dim": 300,
    "ffn_num_layers": 2,
    "message_hidden_dim": 300,
    "dropout": 0.067396
}

In [7]:
def train_and_eval(input_paths, save_dirs, run_idx):
    for input_path, save_dir in zip(input_paths, save_dirs):
        print(f"Training and predicting on {input_path}")
        input_df = pd.read_csv(input_path)
        train_dset, val_dset, pred_dset = prepare_data(input_df)
        model = train_model(MODEL_CONFIG, train_dset, val_dset, NUM_WORKERS, save_dir, run_idx)
        preds = predict(model, pred_dset, NUM_WORKERS)

        output_df = input_df.copy()
        output_df[["pred_" + t for t in TARGET_COLUMNS]] = preds
        save_dir.mkdir(parents=True, exist_ok=True)
        output_df.to_csv(save_dir / "predictions.csv", index=False)

        train_preds = extract_preds(output_df[input_df["split"] == "train"])
        train_refs = extract_refs(input_df[input_df["split"] == "train"])
        val_preds = extract_preds(output_df[input_df["split"] == "val"])
        val_refs = extract_refs(input_df[input_df["split"] == "val"])

        metrics = eval_admet(train_preds, train_refs)
        print("Train metrics:")
        print(json.dumps(metrics, indent=2))

        metrics = eval_admet(val_preds, val_refs)
        print("\nVal metrics:")
        print(json.dumps(metrics, indent=2))
    
    wandb.finish(exit_code=0)

## Run 0

In [8]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_0/split_{k}') for k in range(5)]
RUN_IDX = 0

In [9]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../data/asap/datasets/rnd_splits/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vladvin111 (vladvin-org) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Path ../wandb/chemprop_run_0/wandb/ wasn't writable, using system temp directory.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.13976095772579625,
    "r2": 0.7745557846604791
  },
  "MLM": {
    "mean_absolute_error": 0.19280357091722433,
    "r2": 0.8336789460638887
  },
  "KSOL": {
    "mean_absolute_error": 0.17906946000880447,
    "r2": 0.8744394341862509
  },
  "HLM": {
    "mean_absolute_error": 0.1902789198244699,
    "r2": 0.8252235089309581
  },
  "LogD": {
    "mean_absolute_error": 0.23461366682259024,
    "r2": 0.934951522554137
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.18730531505977704,
    "macro_r2": 0.8485698392791428
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.25306675856993904,
    "r2": 0.3836420110904095
  },
  "MLM": {
    "mean_absolute_error": 0.34520219482359343,
    "r2": 0.5266217678207864
  },
  "KSOL": {
    "mean_absolute_error": 0.38657050695149314,
    "r2": 0.347397043691125
  },
  "HLM": {
    "mean_absolute_error": 0.3542786259767365,
    "r2": 0.25744649271245446
  },
 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.13091335815303531,
    "r2": 0.8163768539514366
  },
  "MLM": {
    "mean_absolute_error": 0.18431139917951317,
    "r2": 0.863770040479959
  },
  "KSOL": {
    "mean_absolute_error": 0.16438884322382957,
    "r2": 0.8783871190030891
  },
  "HLM": {
    "mean_absolute_error": 0.18316722770020522,
    "r2": 0.8315136876984703
  },
  "LogD": {
    "mean_absolute_error": 0.21663748613282624,
    "r2": 0.9480370047230205
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.17588366287788187,
    "macro_r2": 0.8676169411711951
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.19748339039903406,
    "r2": 0.4912223109318955
  },
  "MLM": {
    "mean_absolute_error": 0.3557867512380401,
    "r2": 0.314986081293109
  },
  "KSOL": {
    "mean_absolute_error": 0.2618980014902131,
    "r2": 0.6491954916419501
  },
  "HLM": {
    "mean_absolute_error": 0.33239926832717487,
    "r2": 0.37048821585894787
  },
 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.12748195483936398,
    "r2": 0.8184510547977495
  },
  "MLM": {
    "mean_absolute_error": 0.1604041829021063,
    "r2": 0.9054748994947864
  },
  "KSOL": {
    "mean_absolute_error": 0.17630892981357885,
    "r2": 0.8772759908442778
  },
  "HLM": {
    "mean_absolute_error": 0.16579196263585522,
    "r2": 0.8715346177577278
  },
  "LogD": {
    "mean_absolute_error": 0.19437164396971696,
    "r2": 0.9631249318169899
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.16487173483212425,
    "macro_r2": 0.8871722989423063
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.1664036562219293,
    "r2": 0.67321751419483
  },
  "MLM": {
    "mean_absolute_error": 0.3346573259455529,
    "r2": 0.47036111614434595
  },
  "KSOL": {
    "mean_absolute_error": 0.3056664440928063,
    "r2": 0.5286006613303338
  },
  "HLM": {
    "mean_absolute_error": 0.2697378303878127,
    "r2": 0.5004912066863171
  },
  "L

split_0-epoch,▁▁▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
split_0-train_loss_epoch,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,▅▅█▃▃▅▂▂▃▂▃▂▂▂▅▂▁▂▂▂▁▁▁▁
split_0-val/mae,█▅▅▄▃▂▂▂▂▁▃▃▃▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▇▆▇▆█▇█▇██▇▇▇████▇█████████████████████
split_0-val_loss,█▃▄▄▂▄▂▁▁▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇████
split_1-train_loss_epoch,█▅▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▆█▄▄▇▃▂▆▂▂▄▂▂▃▂▁▅▁▁▂▁▁▆
split_1-val/mae,▃▆▄▄█▅█▂▂▃▂▂▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▁▆▇▆▇▅▃▄▇▇██████████████████████████████


## Cleaning up + run 1

In [10]:
def clean_data(input_paths, save_dirs, output_dir, remove_worst_pct):
    smiles_to_remove = defaultdict(set)

    for input_path, save_dir in zip(input_paths, save_dirs):
        input_df = pd.read_csv(input_path)
        input_val_df = input_df[input_df["split"] == "val"]
        output_df = pd.read_csv(save_dir / "predictions.csv")
        output_val_df = output_df[input_df["split"] == "val"]

        for t in TARGET_COLUMNS:
            # Sort by absolute error
            notna_mask = input_val_df[t].notna()
            input_val_df = input_val_df[notna_mask]
            output_val_df = output_val_df[notna_mask]

            mae = np.abs(input_val_df[t] - output_val_df[f"pred_{t}"])
            sorted_idx = np.argsort(mae)[::-1]
            smiles_to_remove[t].update(
                input_val_df.iloc[sorted_idx[:int(remove_worst_pct * len(sorted_idx))]]["cxsmiles_std"].tolist()
            )

    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[input_df["cxsmiles_std"].isin(smiles_to_remove[t]) & (input_df["split"] == "train"), t] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [11]:
output_dir = Path("../output/asap/rnd_splits/chemprop/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [12]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [13]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            150
LogMLM            140
LogD               86
LogKSOL            74
LogMDR1-MDCKII     15
dtype: int64

In [14]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            189
LogMLM            173
LogD              116
LogKSOL            99
LogMDR1-MDCKII     40
dtype: int64

In [15]:
input_paths = [Path(f'../output/asap/rnd_splits/chemprop/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_worst_pct_0.2"

In [16]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../output/asap/rnd_splits/chemprop/run_0/cleaned/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/chemprop_run_1_clean_worst_pct_0.2/wandb/ wasn't writable, using system temp directory.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.14227374236841633,
    "r2": 0.7353540115490398
  },
  "MLM": {
    "mean_absolute_error": 0.2963343943622467,
    "r2": 0.5477432507973682
  },
  "KSOL": {
    "mean_absolute_error": 0.20010252492207506,
    "r2": 0.7769310188631199
  },
  "HLM": {
    "mean_absolute_error": 0.23806429657130881,
    "r2": 0.6408682658877078
  },
  "LogD": {
    "mean_absolute_error": 0.22839638884331095,
    "r2": 0.9418078026722074
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.22103426941347157,
    "macro_r2": 0.7285408699538886
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.23614128593589734,
    "r2": 0.40299723410658284
  },
  "MLM": {
    "mean_absolute_error": 0.3484029686636332,
    "r2": 0.54000280898417
  },
  "KSOL": {
    "mean_absolute_error": 0.36301804071797455,
    "r2": 0.4130066676453735
  },
  "HLM": {
    "mean_absolute_error": 0.29240699125891917,
    "r2": 0.5041403711202228
  },
 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.15103762620646247,
    "r2": 0.745775461897638
  },
  "MLM": {
    "mean_absolute_error": 0.2693297661042635,
    "r2": 0.6460844221756252
  },
  "KSOL": {
    "mean_absolute_error": 0.23586322843694266,
    "r2": 0.7523196838199121
  },
  "HLM": {
    "mean_absolute_error": 0.24472650847591101,
    "r2": 0.6071729531154877
  },
  "LogD": {
    "mean_absolute_error": 0.23151375632315477,
    "r2": 0.9367832902111477
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.22649417710934688,
    "macro_r2": 0.7376271622439622
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.18988518697222675,
    "r2": 0.541743699556463
  },
  "MLM": {
    "mean_absolute_error": 0.3375705479144024,
    "r2": 0.39275333991517125
  },
  "KSOL": {
    "mean_absolute_error": 0.3197205475636028,
    "r2": 0.5379962201517265
  },
  "HLM": {
    "mean_absolute_error": 0.304955035005082,
    "r2": 0.5605224259199488
  },
  "L

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.12883654904531794,
    "r2": 0.7890940722412285
  },
  "MLM": {
    "mean_absolute_error": 0.24882016085974398,
    "r2": 0.6066427326699031
  },
  "KSOL": {
    "mean_absolute_error": 0.22821711122190355,
    "r2": 0.7284967651842527
  },
  "HLM": {
    "mean_absolute_error": 0.23156985534502952,
    "r2": 0.6422582089431625
  },
  "LogD": {
    "mean_absolute_error": 0.17526590539567302,
    "r2": 0.9667749126222479
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.20254191637353364,
    "macro_r2": 0.746653338332159
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.19523747724836107,
    "r2": 0.5765835408067206
  },
  "MLM": {
    "mean_absolute_error": 0.31330708510634375,
    "r2": 0.6117288150794707
  },
  "KSOL": {
    "mean_absolute_error": 0.30763271517541574,
    "r2": 0.5615823859170865
  },
  "HLM": {
    "mean_absolute_error": 0.23535738241792314,
    "r2": 0.6451514093193955
  },

split_0-epoch,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
split_0-train_loss_epoch,█▇██▆▅▆▆▃▃▂▃▃▂▃▃▂▃▂▂▂▂▂▂▁▁▂▂▁▁▁▁▂▂▁▂▁▁▁▁
split_0-train_loss_step,▅▄▂▂▃▅▂▂█▁▂▃▁▁▄▁▁▂▁▁▃▁▁▂
split_0-val/mae,▇▃▄▅██▅▂▃▃▂▁▁▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▆▇▅▆▇▇▇▇██████████▇████████████████████
split_0-val_loss,▅▄▃▅▅▄█▆█▂▃▂▂▂▁▁▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇█
split_1-train_loss_epoch,█▇▆▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁
split_1-train_loss_step,▅▃▇▃▃█▂▃▃▂▂▃▂▂▁▂▁▂▁▁▂▁▂▆
split_1-val/mae,▄▄▆█▄▅▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▆▇██▇▁█▇▇███▇███████████████████████████


## Cleaning up + run 2

In [17]:
output_dir = Path("../output/asap/rnd_splits/chemprop/run_1/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [18]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [19]:
input_paths = [Path(f'../output/asap/rnd_splits/chemprop/run_1/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_2/split_{k}') for k in range(5)]
RUN_IDX = "2_clean_worst_pct_0.2"

In [20]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../output/asap/rnd_splits/chemprop/run_1/cleaned/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/chemprop_run_2_clean_worst_pct_0.2/wandb/ wasn't writable, using system temp directory.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.14433189857263778,
    "r2": 0.7343117646599389
  },
  "MLM": {
    "mean_absolute_error": 0.3030723889076558,
    "r2": 0.4483996284114614
  },
  "KSOL": {
    "mean_absolute_error": 0.22286873052930775,
    "r2": 0.7472875128411507
  },
  "HLM": {
    "mean_absolute_error": 0.25181634740533604,
    "r2": 0.5895968588059259
  },
  "LogD": {
    "mean_absolute_error": 0.20207323176560008,
    "r2": 0.9493166304981331
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.22483251943610746,
    "macro_r2": 0.693782479043322
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.22873153947447775,
    "r2": 0.43265989790062076
  },
  "MLM": {
    "mean_absolute_error": 0.3550933088815644,
    "r2": 0.5150682122556909
  },
  "KSOL": {
    "mean_absolute_error": 0.3590109339954725,
    "r2": 0.4399750711998729
  },
  "HLM": {
    "mean_absolute_error": 0.3587496459248311,
    "r2": 0.17323386885672987
  },
 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.1548917673412998,
    "r2": 0.6709189931830133
  },
  "MLM": {
    "mean_absolute_error": 0.2813709544235244,
    "r2": 0.569166414642083
  },
  "KSOL": {
    "mean_absolute_error": 0.2540727243751764,
    "r2": 0.6474226394193212
  },
  "HLM": {
    "mean_absolute_error": 0.2600090589116916,
    "r2": 0.5529598410207885
  },
  "LogD": {
    "mean_absolute_error": 0.2087018375232886,
    "r2": 0.9520625312987836
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.23180926851499617,
    "macro_r2": 0.6785060839127979
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.2197529837750959,
    "r2": 0.34154701772806006
  },
  "MLM": {
    "mean_absolute_error": 0.3584255566538461,
    "r2": 0.26186949693383843
  },
  "KSOL": {
    "mean_absolute_error": 0.2809791993639221,
    "r2": 0.6137602925277275
  },
  "HLM": {
    "mean_absolute_error": 0.33719118977328505,
    "r2": 0.42558485669890234
  },
  "L

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.1276227171024837,
    "r2": 0.7699295783318014
  },
  "MLM": {
    "mean_absolute_error": 0.27578298429831866,
    "r2": 0.5402087708170444
  },
  "KSOL": {
    "mean_absolute_error": 0.22807806656505117,
    "r2": 0.6601921076692168
  },
  "HLM": {
    "mean_absolute_error": 0.26324956121950444,
    "r2": 0.5280428316085671
  },
  "LogD": {
    "mean_absolute_error": 0.17978504606180415,
    "r2": 0.9656460663793344
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.21490367504943242,
    "macro_r2": 0.6928038709611928
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.19424659523178953,
    "r2": 0.5702766369291481
  },
  "MLM": {
    "mean_absolute_error": 0.321882875949617,
    "r2": 0.5338286182110992
  },
  "KSOL": {
    "mean_absolute_error": 0.28230069505449235,
    "r2": 0.6193218828477111
  },
  "HLM": {
    "mean_absolute_error": 0.28827899872860724,
    "r2": 0.44117234571190334
  },


split_0-epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇██
split_0-train_loss_epoch,█▇▅▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,▅█▂▂▃█▂▂█▂▂▂▂▂▃▁▁▅▁▁▃▁▁▂
split_0-val/mae,▇▆▂▆█▂▄▂▂▃▂▂▂▁▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▇▄▇▇██▅▇███▇███████████████████████████
split_0-val_loss,█▃▂▂▃▂▃▂▂▂▂▅▂▂▂▃▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
split_1-train_loss_epoch,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,▄▃▃▂▂█▂▂▃▂▂▂▁▂▁▁▁▂▁▁▁▁▁▂
split_1-val/mae,█▂▅▃▅▄▄▂▂▃▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▅▁▄▄▂▅▅▄▄▆▇▇███▇▇▇███████▇██████████████


## Cleaning up stero impure + run 1

In [21]:
def clean_data(input_paths, save_dirs, output_dir, remove_worst_pct):
    smiles_to_remove = defaultdict(set)

    for input_path, save_dir in zip(input_paths, save_dirs):
        input_df = pd.read_csv(input_path)
        input_val_df = input_df[input_df["split"] == "val"]
        output_df = pd.read_csv(save_dir / "predictions.csv")
        output_val_df = output_df[input_df["split"] == "val"]

        for t in TARGET_COLUMNS:
            # Sort by absolute error
            notna_mask = input_val_df[t].notna()
            input_val_df = input_val_df[notna_mask]
            output_val_df = output_val_df[notna_mask]

            mae = np.abs(input_val_df[t] - output_val_df[f"pred_{t}"])
            sorted_idx = np.argsort(mae)[::-1]
            smiles_to_remove[t].update(
                input_val_df.iloc[sorted_idx[:int(remove_worst_pct * len(sorted_idx))]]["cxsmiles_std"].tolist()
            )

    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[
                input_df["cxsmiles_std"].isin(smiles_to_remove[t]) & \
                    ~input_df["smiles_ext"].isna() & \
                    (input_df["split"] == "train"),
                t
            ] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [22]:
output_dir = Path("../output/asap/rnd_splits/chemprop/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [23]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [24]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            205
LogMLM            183
LogD              129
LogKSOL           111
LogMDR1-MDCKII     49
dtype: int64

In [25]:
tmp = pd.read_csv(input_paths[0])
tmp = tmp[tmp["smiles_ext"].isna()]
tmp[TARGET_COLUMNS].isna().sum()

LogHLM            103
LogMLM             85
LogD               69
LogKSOL            58
LogMDR1-MDCKII     29
dtype: int64

In [26]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            210
LogMLM            184
LogD              133
LogKSOL           111
LogMDR1-MDCKII     51
dtype: int64

In [27]:
tmp = pd.read_csv(output_dir / input_paths[0].name)
tmp = tmp[tmp["smiles_ext"].isna()]
tmp[TARGET_COLUMNS].isna().sum()

LogHLM            103
LogMLM             85
LogD               69
LogKSOL            58
LogMDR1-MDCKII     29
dtype: int64

In [28]:
input_paths = [Path(f'../output/asap/rnd_splits/chemprop/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_worst_pct_0.2_stereo_impure"

In [29]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../output/asap/rnd_splits/chemprop/run_0/cleaned/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/chemprop_run_1_clean_worst_pct_0.2_stereo_impure/wandb/ wasn't writable, using system temp directory.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.14554679789818198,
    "r2": 0.7209943060027302
  },
  "MLM": {
    "mean_absolute_error": 0.29848413293019055,
    "r2": 0.48393688297026005
  },
  "KSOL": {
    "mean_absolute_error": 0.21659086944821093,
    "r2": 0.7471004770898493
  },
  "HLM": {
    "mean_absolute_error": 0.256705934317879,
    "r2": 0.5783958964649301
  },
  "LogD": {
    "mean_absolute_error": 0.21188328144876728,
    "r2": 0.9473113397818473
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.22584220320864595,
    "macro_r2": 0.6955477804619233
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.23627959414627958,
    "r2": 0.32402941934441953
  },
  "MLM": {
    "mean_absolute_error": 0.3497851386278697,
    "r2": 0.5183510969415172
  },
  "KSOL": {
    "mean_absolute_error": 0.3808477378953188,
    "r2": 0.3236927753160128
  },
  "HLM": {
    "mean_absolute_error": 0.3509671582599759,
    "r2": 0.26124095380280965
  },


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.1606516078227226,
    "r2": 0.6651822797475369
  },
  "MLM": {
    "mean_absolute_error": 0.3126021038296431,
    "r2": 0.5216193136325642
  },
  "KSOL": {
    "mean_absolute_error": 0.23092579951299105,
    "r2": 0.6942086747755813
  },
  "HLM": {
    "mean_absolute_error": 0.2829410938545044,
    "r2": 0.49263015081287587
  },
  "LogD": {
    "mean_absolute_error": 0.18260126922279593,
    "r2": 0.9607218675103385
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.2339443748485314,
    "macro_r2": 0.6668724572957794
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.20820052107684664,
    "r2": 0.3737095564179833
  },
  "MLM": {
    "mean_absolute_error": 0.36120826602653183,
    "r2": 0.2535580313736424
  },
  "KSOL": {
    "mean_absolute_error": 0.25299842822524393,
    "r2": 0.6726996768871472
  },
  "HLM": {
    "mean_absolute_error": 0.36651812013425905,
    "r2": 0.33835094427517975
  },


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.14009077730370453,
    "r2": 0.7205998621284329
  },
  "MLM": {
    "mean_absolute_error": 0.2863904568101407,
    "r2": 0.5027252082289837
  },
  "KSOL": {
    "mean_absolute_error": 0.2351839849059592,
    "r2": 0.6512215949545133
  },
  "HLM": {
    "mean_absolute_error": 0.26924683620039,
    "r2": 0.5345756214957287
  },
  "LogD": {
    "mean_absolute_error": 0.19807500775833176,
    "r2": 0.9568606574167434
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.22579741259570527,
    "macro_r2": 0.6731965888448804
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.2008872510731234,
    "r2": 0.5389876598702514
  },
  "MLM": {
    "mean_absolute_error": 0.3488343767333001,
    "r2": 0.42726357592285835
  },
  "KSOL": {
    "mean_absolute_error": 0.2877768033340913,
    "r2": 0.6136183927741916
  },
  "HLM": {
    "mean_absolute_error": 0.2934634862834378,
    "r2": 0.4353177586517786
  },
  "Log

split_0-epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
split_0-train_loss_epoch,█▆▆▆▅▄▄▃▂▂▃▃▃▂▂▂▃▃▂▂▂▂▂▁▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
split_0-train_loss_step,▇▃▄▂▄▂▂▂▃▂▁▃▂▁▇▁▁▃▁▁▁▁▁█
split_0-val/mae,█▃▃▃▃▄▃▂▂▅▂▁▂▂▁▂▂▂▂▂▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▆▃▅▆▅▂▇▆▇███▇██████▇███████████████████
split_0-val_loss,█▃▂▃▅▆▃▄▂▁▂▃▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
split_1-train_loss_epoch,█▇▆▆▅▅▅▄▄▄▃▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▆▂▃▃▄▂▂▅▂▃▂▂▁▆▁▁▁▁▁▃▁▁▃
split_1-val/mae,▄▂▂▇▆█▃▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▆▇▆▅▁▇██▇▇██████████████████████████████


## Removing all stereo impure + run 0

In [30]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
output_dir = Path("../data/asap/datasets/rnd_splits/stereo_pure")
output_dir.mkdir(parents=True, exist_ok=True)

for input_path in input_paths:
    input_df = pd.read_csv(input_path)
    input_df = pd.concat([
        input_df[(input_df["split"] == "train") & input_df["smiles_ext"].isna()],
        input_df[input_df["split"] == "val"]
    ])
    input_df.to_csv(output_dir / input_path.name, index=False)

In [31]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/stereo_pure/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_0/stereo_pure/split_{k}') for k in range(5)]
RUN_IDX = "0_stereo_pure"
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../data/asap/datasets/rnd_splits/stereo_pure/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/chemprop_run_0_stereo_pure/wandb/ wasn't writable, using system temp directory.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.09081783103687854,
    "r2": 0.8958436462874637
  },
  "MLM": {
    "mean_absolute_error": 0.10608101840994032,
    "r2": 0.9595544873004163
  },
  "KSOL": {
    "mean_absolute_error": 0.10652891766395049,
    "r2": 0.9644793622153466
  },
  "HLM": {
    "mean_absolute_error": 0.11375989068184703,
    "r2": 0.9434973283358553
  },
  "LogD": {
    "mean_absolute_error": 0.10852208788234453,
    "r2": 0.9880827345388887
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.10514194913499217,
    "macro_r2": 0.950291511735594
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.2827059181814536,
    "r2": 0.26113799002809335
  },
  "MLM": {
    "mean_absolute_error": 0.4806832282628319,
    "r2": 0.1633109312619202
  },
  "KSOL": {
    "mean_absolute_error": 0.5264107479772971,
    "r2": -0.04257101940870567
  },
  "HLM": {
    "mean_absolute_error": 0.45862773882981306,
    "r2": -0.21029553655285027
  

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.0878478433079328,
    "r2": 0.9063431019985975
  },
  "MLM": {
    "mean_absolute_error": 0.11588565417159902,
    "r2": 0.9490839072057556
  },
  "KSOL": {
    "mean_absolute_error": 0.11764483518399409,
    "r2": 0.9532017095908344
  },
  "HLM": {
    "mean_absolute_error": 0.09300598485972264,
    "r2": 0.9555000204432179
  },
  "LogD": {
    "mean_absolute_error": 0.12791373340910625,
    "r2": 0.9848294781286626
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.10845961018647096,
    "macro_r2": 0.9497916434734137
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.23487879354711622,
    "r2": 0.22673695992563692
  },
  "MLM": {
    "mean_absolute_error": 0.4949576875077337,
    "r2": -0.15891725348953378
  },
  "KSOL": {
    "mean_absolute_error": 0.5483204526969018,
    "r2": -0.3223996178247015
  },
  "HLM": {
    "mean_absolute_error": 0.47564855809098433,
    "r2": -0.006948656871571979

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.0937570134301367,
    "r2": 0.8972990863711714
  },
  "MLM": {
    "mean_absolute_error": 0.12287498886632471,
    "r2": 0.9499805886594006
  },
  "KSOL": {
    "mean_absolute_error": 0.09333903999893156,
    "r2": 0.964389612571935
  },
  "HLM": {
    "mean_absolute_error": 0.12048900477289336,
    "r2": 0.9472300066649776
  },
  "LogD": {
    "mean_absolute_error": 0.14933197603438253,
    "r2": 0.9794325030365834
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.11595840462053378,
    "macro_r2": 0.9476663594608136
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.2624842362576178,
    "r2": 0.32425688284831977
  },
  "MLM": {
    "mean_absolute_error": 0.4285446999095742,
    "r2": 0.1175267970194489
  },
  "KSOL": {
    "mean_absolute_error": 0.4746975299984798,
    "r2": 0.05367089807029879
  },
  "HLM": {
    "mean_absolute_error": 0.36438465714082696,
    "r2": 0.12104900664796359
  },


split_0-epoch,▁▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇█████
split_0-train_loss_epoch,█▇▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▄▄▂▁▁▁▂▁▁▁▁
split_0-val/mae,█▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▃▄▄▅▆▇▇████████████████████████████████
split_0-val_loss,█▃▃▂▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇██
split_1-train_loss_epoch,██▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▄▂▂▂▂▂▁▁▁▁▁
split_1-val/mae,█▇▆▅▆▃▂▂▁▂▂▂▁▂▂▂▁▂▂▁▁▂▁▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▁▆▆▇▇███████████████████████████████████


## Cleaning up unstable predictions for stero impure + run 1

In [32]:
input_df = pd.read_csv("../data/asap/datasets/rnd_splits/split_0.csv")
input_df

,smiles,HLM,KSOL,LogD,MLM,MDR1-MDCKII,smiles_std,cxsmiles_std,mol_idx,smiles_ext,LogHLM,LogMLM,LogKSOL,LogMDR1-MDCKII,split
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,NaN,NaN,0.3,NaN,2.0,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1 |a:16|,191,|a:16|,NaN,NaN,NaN,0.477121,val
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,NaN,333.0,2.9,NaN,0.2,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...,335,|&1:7|,NaN,NaN,2.523746,0.079181,train
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,NaN,NaN,0.4,NaN,0.5,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...,336,|&1:7|,NaN,NaN,NaN,0.176091,train
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,NaN,376.0,1.0,NaN,8.5,NC(=O)[C@H]1CCCN(C(=O)Cc2cccc3occc23)C1,NC(=O)[C@H]1CCCN(C(=O)Cc2cccc3occc23)C1 |&1:3|,300,|&1:3|,NaN,NaN,2.576341,0.977724,train
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,NaN,375.0,-0.3,NaN,0.9,Cc1cc(CC(=O)N2CCC[C@H](C(N)=O)C2)ccn1,Cc1cc(CC(=O)N2CCC[C@H](C(N)=O)C2)ccn1 |&1:11|,249,|&1:11|,NaN,NaN,2.575188,0.278754,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,CC(C)NC[C@H](O)COC1=CC=CC2=CC=CC=C12 |&1:5|,25.5,NaN,NaN,63.0,NaN,CC(C)NC[C@H](O)COc1cccc2ccccc12,CC(C)NC[C@H](O)COc1cccc2ccccc12 |&1:5|,22,|&1:5|,1.423246,1.806180,NaN,NaN,val
400,O=C(O)CC1=CC=CC=C1NC1=C(Cl)C=CC=C1Cl,216.0,NaN,NaN,386.0,NaN,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl,380,NaN,2.336460,2.587711,NaN,NaN,val
401,NCC1=CC(Cl)=CC(C(=O)NC2=CC=C3CNCC3=C2)=C1,NaN,NaN,2.0,NaN,NaN,NCc1cc(Cl)cc(C(=O)Nc2ccc3c(c2)CNC3)c1,NCc1cc(Cl)cc(C(=O)Nc2ccc3c(c2)CNC3)c1,303,NaN,NaN,NaN,NaN,NaN,train
402,COC(=O)NC1=NC2=CC=C(C(=O)C3=CC=CC=C3)C=C2N1,NaN,NaN,2.9,NaN,NaN,COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1,COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1,166,NaN,NaN,NaN,NaN,NaN,train


In [33]:
input_df["mol_idx"].nunique()

404

In [34]:
from rdkit import Chem
from rdkit.Chem import EnumerateStereoisomers
opts = EnumerateStereoisomers.StereoEnumerationOptions()
opts.onlyStereoGroups = True

In [35]:
def get_all_isomers(smiles: str) -> list[str]:
    mol = Chem.MolFromSmiles(smiles)
    enumerated = EnumerateStereoisomers.EnumerateStereoisomers(mol, opts)

    return [Chem.MolToSmiles(isomer) for isomer in enumerated]

def calculate_num_isomers(input_df: pd.DataFrame):
    input_df = input_df.copy()
    input_df["n_isomers"] = input_df["cxsmiles_std"].apply(lambda x: len(get_all_isomers(x)))
    
    return input_df

In [36]:
input_df = calculate_num_isomers(input_df)
input_df

,smiles,HLM,KSOL,LogD,MLM,MDR1-MDCKII,smiles_std,cxsmiles_std,mol_idx,smiles_ext,LogHLM,LogMLM,LogKSOL,LogMDR1-MDCKII,split,n_isomers
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,NaN,NaN,0.3,NaN,2.0,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1 |a:16|,191,|a:16|,NaN,NaN,NaN,0.477121,val,1
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,NaN,333.0,2.9,NaN,0.2,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...,335,|&1:7|,NaN,NaN,2.523746,0.079181,train,2
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,NaN,NaN,0.4,NaN,0.5,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...,336,|&1:7|,NaN,NaN,NaN,0.176091,train,2
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,NaN,376.0,1.0,NaN,8.5,NC(=O)[C@H]1CCCN(C(=O)Cc2cccc3occc23)C1,NC(=O)[C@H]1CCCN(C(=O)Cc2cccc3occc23)C1 |&1:3|,300,|&1:3|,NaN,NaN,2.576341,0.977724,train,2
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,NaN,375.0,-0.3,NaN,0.9,Cc1cc(CC(=O)N2CCC[C@H](C(N)=O)C2)ccn1,Cc1cc(CC(=O)N2CCC[C@H](C(N)=O)C2)ccn1 |&1:11|,249,|&1:11|,NaN,NaN,2.575188,0.278754,train,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,CC(C)NC[C@H](O)COC1=CC=CC2=CC=CC=C12 |&1:5|,25.5,NaN,NaN,63.0,NaN,CC(C)NC[C@H](O)COc1cccc2ccccc12,CC(C)NC[C@H](O)COc1cccc2ccccc12 |&1:5|,22,|&1:5|,1.423246,1.806180,NaN,NaN,val,2
400,O=C(O)CC1=CC=CC=C1NC1=C(Cl)C=CC=C1Cl,216.0,NaN,NaN,386.0,NaN,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl,380,NaN,2.336460,2.587711,NaN,NaN,val,1
401,NCC1=CC(Cl)=CC(C(=O)NC2=CC=C3CNCC3=C2)=C1,NaN,NaN,2.0,NaN,NaN,NCc1cc(Cl)cc(C(=O)Nc2ccc3c(c2)CNC3)c1,NCc1cc(Cl)cc(C(=O)Nc2ccc3c(c2)CNC3)c1,303,NaN,NaN,NaN,NaN,NaN,train,1
402,COC(=O)NC1=NC2=CC=C(C(=O)C3=CC=CC=C3)C=C2N1,NaN,NaN,2.9,NaN,NaN,COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1,COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1,166,NaN,NaN,NaN,NaN,NaN,train,1


In [37]:
input_df[(input_df["n_isomers"] > 1) & (input_df["smiles_ext"].isna())]

,smiles,HLM,KSOL,LogD,MLM,MDR1-MDCKII,smiles_std,cxsmiles_std,mol_idx,smiles_ext,LogHLM,LogMLM,LogKSOL,LogMDR1-MDCKII,split,n_isomers


In [38]:
tmp = input_df[~input_df["smiles_ext"].isna()]
tmp[(tmp["n_isomers"] > 1) & (tmp["smiles_ext"].str.contains("a"))]

,smiles,HLM,KSOL,LogD,MLM,MDR1-MDCKII,smiles_std,cxsmiles_std,mol_idx,smiles_ext,LogHLM,LogMLM,LogKSOL,LogMDR1-MDCKII,split,n_isomers


In [39]:
tmp["n_isomers"].value_counts()

n_isomers
2    170
1     20
4      9
Name: count, dtype: int64

In [40]:
tmp[tmp["smiles_ext"].str.contains("a")]

,smiles,HLM,KSOL,LogD,MLM,MDR1-MDCKII,smiles_std,cxsmiles_std,mol_idx,smiles_ext,LogHLM,LogMLM,LogKSOL,LogMDR1-MDCKII,split,n_isomers
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,NaN,NaN,0.3,NaN,2.000,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1 |a:16|,191,|a:16|,NaN,NaN,NaN,0.477121,val,1
51,O=C(CC1=CN=CC2=CC=CC=C12)N1CC[C@@H](OC2=CC=C(F...,NaN,324.0,NaN,NaN,21.700,O=C(Cc1cncc2ccccc12)N1CC[C@@H](Oc2ccc(F)cc2)C1,O=C(Cc1cncc2ccccc12)N1CC[C@@H](Oc2ccc(F)cc2)C1...,326,|a:16|,NaN,NaN,2.511883,1.356026,train,1
175,CC1=CC(C2=NOC(C(F)(F)F)=N2)=CC=C1OCCCC1=CC(C(=...,NaN,12.0,4.4,13.0,2.090,Cc1cc(-c2noc(C(F)(F)F)n2)ccc1OCCCc1cc(C(=O)N2C...,Cc1cc(-c2noc(C(F)(F)F)n2)ccc1OCCCc1cc(C(=O)N2C...,231,|a:28|,NaN,1.146128,1.113943,0.489958,train,1
193,CC1=CC(C2=NOC(C(F)(F)F)=N2)=CC=C1OCCCC1=CC(C(=...,NaN,13.0,4.4,12.3,2.470,Cc1cc(-c2noc(C(F)(F)F)n2)ccc1OCCCc1cc(C(=O)N2C...,Cc1cc(-c2noc(C(F)(F)F)n2)ccc1OCCCc1cc(C(=O)N2C...,232,|a:28|,NaN,1.123852,1.146128,0.540329,train,1
195,O=C1NC(=O)[C@@H](CC2=CC=C3OCOC3=C2)C2=CC=CC=C1...,32.4,263.0,2.4,82.7,6.560,O=C1NC(=O)[C@@H](Cc2ccc3c(c2)OCO3)c2ccccc21,O=C1NC(=O)[C@@H](Cc2ccc3c(c2)OCO3)c2ccccc21 |a:5|,391,|a:5|,1.523746,1.922725,2.421604,0.878522,val,1
225,NCC1=CC=CC(NC(=O)[C@@H](NC(=O)OCC2=CC=CC=C2)C2...,11.0,23.0,3.2,1.0,1.530,NCc1cccc(NC(=O)[C@@H](NC(=O)OCc2ccccc2)c2ccc(O...,NCc1cccc(NC(=O)[C@@H](NC(=O)OCc2ccccc2)c2ccc(O...,308,|a:10|,1.079181,0.301030,1.380211,0.403121,train,1
241,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,19.0,NaN,1.2,57.0,1.000,C[C@H]1CN(c2cncc3ccccc23)C(=O)[C@@]12CN(Cc1ccn...,C[C@H]1CN(c2cncc3ccccc23)C(=O)[C@@]12CN(Cc1ccn...,214,"|a:1,16|",1.301030,1.763428,NaN,0.301030,train,1
242,CNC(=O)CN1C[C@@]2(C(=O)N(C3=CN=CC4=CC=CC=C34)C...,70.0,390.0,2.1,75.0,0.500,CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H...,CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H...,115,"|a:7,22|",1.851258,1.880814,2.592177,0.176091,train,1
243,CNC(=O)CN1C[C@@]2(C(=O)N(C3=CN=CC4=CC=CC=C34)C...,39.0,397.0,1.8,77.0,0.600,CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H...,CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H...,120,"|a:7,22|",1.602060,1.892095,2.599883,0.204120,train,1
244,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,237.0,376.0,1.8,237.0,1.330,C[C@H]1CN(c2cncc3ccccc23)C(=O)[C@@]12CN(CCN1CC...,C[C@H]1CN(c2cncc3ccccc23)C(=O)[C@@]12CN(CCN1CC...,206,"|a:1,16|",2.376577,2.376577,2.576341,0.367356,train,1


In [41]:
def enumerate_isomers(input_df: pd.DataFrame):
    input_df = input_df.copy()
    # we don't want to find isomers for compounds with absolute stereochemistry
    # so exlude them and just leave the original compound as the only isomer
    input_df["cxsmiles_std_isomer"] = input_df["cxsmiles_std"]
    input_df.loc[input_df["smiles_ext"].isna(), "smiles_ext"] = ""
    stereo_impure_mask = ~input_df["smiles_ext"].isna() & ~input_df["smiles_ext"].str.contains("a")
    input_df.loc[stereo_impure_mask, "cxsmiles_std_isomer"] = \
        input_df.loc[stereo_impure_mask, "cxsmiles_std"].apply(lambda x: get_all_isomers(x))
    
    input_df = input_df.explode("cxsmiles_std_isomer")
    
    return input_df

In [42]:
enumerate_isomers(input_df)

,smiles,HLM,KSOL,LogD,MLM,MDR1-MDCKII,smiles_std,cxsmiles_std,mol_idx,smiles_ext,LogHLM,LogMLM,LogKSOL,LogMDR1-MDCKII,split,n_isomers,cxsmiles_std_isomer
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,NaN,NaN,0.3,NaN,2.0,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1 |a:16|,191,|a:16|,NaN,NaN,NaN,0.477121,val,1,COc1cccc(Cl)c1NC(=O)N1CCC[C@H](C(N)=O)C1 |a:16|
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,NaN,333.0,2.9,NaN,0.2,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...,335,|&1:7|,NaN,NaN,2.523746,0.079181,train,2,O=C(NCC(F)F)[C@@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)...
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,NaN,333.0,2.9,NaN,0.2,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...,335,|&1:7|,NaN,NaN,2.523746,0.079181,train,2,O=C(NCC(F)F)[C@H](Nc1cc2c(cc1Br)CNC2)c1cc(Cl)c...
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,NaN,NaN,0.4,NaN,0.5,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...,336,|&1:7|,NaN,NaN,NaN,0.176091,train,2,O=C(NCC(F)F)[C@@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc...
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,NaN,NaN,0.4,NaN,0.5,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...,336,|&1:7|,NaN,NaN,NaN,0.176091,train,2,O=C(NCC(F)F)[C@H](Nc1ccc2c(c1)CNC2)c1cc(Br)cc2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,O=C(O)CC1=CC=CC=C1NC1=C(Cl)C=CC=C1Cl,216.0,NaN,NaN,386.0,NaN,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl,380,,2.33646,2.587711,NaN,NaN,val,1,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl
401,NCC1=CC(Cl)=CC(C(=O)NC2=CC=C3CNCC3=C2)=C1,NaN,NaN,2.0,NaN,NaN,NCc1cc(Cl)cc(C(=O)Nc2ccc3c(c2)CNC3)c1,NCc1cc(Cl)cc(C(=O)Nc2ccc3c(c2)CNC3)c1,303,,NaN,NaN,NaN,NaN,train,1,NCc1cc(Cl)cc(C(=O)Nc2ccc3c(c2)CNC3)c1
402,COC(=O)NC1=NC2=CC=C(C(=O)C3=CC=CC=C3)C=C2N1,NaN,NaN,2.9,NaN,NaN,COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1,COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1,166,,NaN,NaN,NaN,NaN,train,1,COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1
403,CC1=NC=CN1C[C@H]1CCC2=C(C1=O)C1=CC=CC=C1N2C |&...,NaN,127.0,NaN,NaN,NaN,Cc1nccn1C[C@H]1CCc2c(c3ccccc3n2C)C1=O,Cc1nccn1C[C@H]1CCc2c(c3ccccc3n2C)C1=O |&1:7|,268,|&1:7|,NaN,NaN,2.107210,NaN,train,2,Cc1nccn1C[C@@H]1CCc2c(c3ccccc3n2C)C1=O


In [43]:
PROPERTY2STD = {
    "LogHLM": 217.0507262281013,
    "LogMLM": 318.4791749144294,
    "LogD": 1.2280921175245259,
    "LogKSOL": 157.6780176767439,
    "LogMDR1-MDCKII": 6.433744322214307
}

def clean_data(input_paths, output_dir):
    mols_to_remove = defaultdict(set)

    for input_path in input_paths:
        print(f"Training and predicting on {input_path}")
        input_df = pd.read_csv(input_path)
        input_df["cxsmiles_std_isomer"] = input_df["cxsmiles_std"]
        input_df = pd.concat([
            input_df[input_df["split"] == "train"],
            enumerate_isomers(input_df[input_df["split"] == "val"])
        ]).reset_index(drop=True)
        train_dset, val_dset, pred_dset = prepare_data(input_df, smiles_column_for_pred="cxsmiles_std_isomer")

        save_dir = output_dir / f"{input_path.stem}"
        model = train_model(MODEL_CONFIG, train_dset, val_dset, NUM_WORKERS, save_dir, run_idx=None, enable_logger=False)
        preds = predict(model, pred_dset, NUM_WORKERS)

        output_df = input_df.copy()
        output_df[["pred_" + t for t in TARGET_COLUMNS]] = preds
        save_dir.mkdir(parents=True, exist_ok=True)
        output_df.to_csv(save_dir / "predictions.csv", index=False)

        output_val_df = output_df[input_df["split"] == "val"].reset_index(drop=True)
        for t in TARGET_COLUMNS:
            if t in ["LogHLM", "LogMLM", "LogKSOL", "LogMDR1-MDCKII"]:
                output_val_df[f"diff_{t}"] = output_val_df.groupby("mol_idx")[f"pred_{t}"].transform(
                    lambda x: np.power(10, x.max()) - np.power(10, x.min())
                )
            else:
                output_val_df[f"diff_{t}"] = output_val_df.groupby("mol_idx")[f"pred_{t}"].transform(
                    lambda x: x.max() - x.min()
                )
            if t == "LogHLM":
                output_val_df[f"noisy_{t}"] = output_val_df[f"diff_{t}"] > 0.2 * PROPERTY2STD[t]
            elif t == "LogMLM":
                output_val_df[f"noisy_{t}"] = output_val_df[f"diff_{t}"] > 0.2 * PROPERTY2STD[t]
            elif t == "LogD":
                output_val_df[f"noisy_{t}"] = output_val_df[f"diff_{t}"] > 0.2  # absolute error
            elif t == "LogKSOL":
                output_val_df[f"noisy_{t}"] = output_val_df[f"diff_{t}"] > 0.2 * PROPERTY2STD[t]
            elif t == "LogMDR1-MDCKII":
                output_val_df[f"noisy_{t}"] = output_val_df[f"diff_{t}"] > 0.2 * PROPERTY2STD[t]
            
            mols_to_remove[t].update(
                output_val_df[output_val_df[f"noisy_{t}"]]["mol_idx"].tolist()
            )

    for t in TARGET_COLUMNS:
        print(f"[{input_path.stem}] Removing {len(mols_to_remove[t])} noisy mols out of each trainig set for {t}")
    
    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[
                input_df["mol_idx"].isin(mols_to_remove[t]) & \
                    (input_df["split"] == "train"),
                t
            ] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [44]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
output_dir = Path("../output/asap/rnd_splits/chemprop/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [45]:
clean_data(input_paths, output_dir)

Training and predicting on ../data/asap/datasets/rnd_splits/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | trai

Training and predicting on ../data/asap/datasets/rnd_splits/split_1.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | trai

Training and predicting on ../data/asap/datasets/rnd_splits/split_2.csv


...

Training and predicting on ../data/asap/datasets/rnd_splits/split_3.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | trai

Training and predicting on ../data/asap/datasets/rnd_splits/split_4.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | trai

[split_4] Removing 24 noisy mols out of each trainig set for LogHLM
[split_4] Removing 19 noisy mols out of each trainig set for LogMLM
[split_4] Removing 51 noisy mols out of each trainig set for LogD
[split_4] Removing 107 noisy mols out of each trainig set for LogKSOL
[split_4] Removing 26 noisy mols out of each trainig set for LogMDR1-MDCKII


In [46]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            150
LogMLM            140
LogD               86
LogKSOL            74
LogMDR1-MDCKII     15
dtype: int64

In [47]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            156
LogMLM            144
LogD              114
LogKSOL           143
LogMDR1-MDCKII     35
dtype: int64

In [48]:
input_paths = [Path(f'../output/asap/rnd_splits/chemprop/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_noisy_stereo_impure"

In [49]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../output/asap/rnd_splits/chemprop/run_0/cleaned/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.1411963681064776,
    "r2": 0.7610463040234314
  },
  "MLM": {
    "mean_absolute_error": 0.20531464519362042,
    "r2": 0.8262092290294405
  },
  "KSOL": {
    "mean_absolute_error": 0.272671366305889,
    "r2": 0.6910990865008488
  },
  "HLM": {
    "mean_absolute_error": 0.18863395745106004,
    "r2": 0.8154478714323808
  },
  "LogD": {
    "mean_absolute_error": 0.2792098335524834,
    "r2": 0.9126604661381659
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.21740523412190607,
    "macro_r2": 0.8012925914248534
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.23829512500200253,
    "r2": 0.40371039985086954
  },
  "MLM": {
    "mean_absolute_error": 0.36374621702421966,
    "r2": 0.46454323070542536
  },
  "KSOL": {
    "mean_absolute_error": 0.5186389489481255,
    "r2": 0.009306541187294015
  },
  "HLM": {
    "mean_absolute_error": 0.3417672095444478,
    "r2": 0.2990566779504741
  },


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.15171120981994718,
    "r2": 0.7408659626242824
  },
  "MLM": {
    "mean_absolute_error": 0.22358932231466164,
    "r2": 0.8037918224516476
  },
  "KSOL": {
    "mean_absolute_error": 0.27411836935631323,
    "r2": 0.6807041016615336
  },
  "HLM": {
    "mean_absolute_error": 0.22037109125506635,
    "r2": 0.7593034029322336
  },
  "LogD": {
    "mean_absolute_error": 0.25688370984978975,
    "r2": 0.9221632924071972
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.2253347405191556,
    "macro_r2": 0.7813657164153789
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.21506348208485102,
    "r2": 0.4403099258858443
  },
  "MLM": {
    "mean_absolute_error": 0.3748884972874948,
    "r2": 0.2588489970359453
  },
  "KSOL": {
    "mean_absolute_error": 0.34764940333950034,
    "r2": 0.4232344821118943
  },
  "HLM": {
    "mean_absolute_error": 0.38747635924495694,
    "r2": 0.25426604261176555
  },

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_

Train metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.13584448137969604,
    "r2": 0.7864291584555924
  },
  "MLM": {
    "mean_absolute_error": 0.17769066860544952,
    "r2": 0.8740933680902283
  },
  "KSOL": {
    "mean_absolute_error": 0.22963716892947378,
    "r2": 0.7743353245293048
  },
  "HLM": {
    "mean_absolute_error": 0.1713481771088388,
    "r2": 0.8465480163579063
  },
  "LogD": {
    "mean_absolute_error": 0.20871476691761776,
    "r2": 0.9568268208022597
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.18464705258821518,
    "macro_r2": 0.8476465376470582
  }
}

Val metrics:
{
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.20931103272826856,
    "r2": 0.5186938415806356
  },
  "MLM": {
    "mean_absolute_error": 0.3170942797562879,
    "r2": 0.5218710040611444
  },
  "KSOL": {
    "mean_absolute_error": 0.3909227827312873,
    "r2": 0.309558575461362
  },
  "HLM": {
    "mean_absolute_error": 0.26493671801896806,
    "r2": 0.5055669230887527
  },
  

split_0-epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇████
split_0-train_loss_epoch,█▆▆▆▆▅▅▄▄▃▂▃▂▂▂▃▃▃▂▂▃▂▂▂▂▂▁▁▂▂▁▁▁▁▂▁▁▁▁▁
split_0-train_loss_step,▃▂▃▂▂▃▂▁▃▂▂▁▁▁▅▁▁▂▁▁█▁▁▂
split_0-val/mae,▂▂▅█▅▂▃▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▃▄▆▇▆██████▇███████████████████████████
split_0-val_loss,▂▂█▇▃▃▄▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
split_1-train_loss_epoch,█▇▇▅▄▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,▇▄▆▃▃▅▃▃▂▂▂▃▂▁█▂▁▃▂▂▃▁▁▅
split_1-val/mae,▅▂▂█▃▂▂▂▂▁▁▁▁▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▅▇▆▁▅▇▆▇▇███▇▇▇█████████████████████████
